<a href="https://colab.research.google.com/github/PaoloGerosa/Chemotherapy-Associated-Liver-Injury/blob/main/Distances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Euclidean Distance

In [ ]:
import math

def euclidean_distance(vol_i, vol_j):
  distance = 0
  for i in range(len(vol_i)):
    for j in range(len(vol_i[0])):
      for z in range(len(vol_i[0][0])):
        distance += (vol_i[i][j][z] - vol_j[i][j][z])**2
  return math.sqrt(distance)

def get_eucl_dist(volumes):
  dist = [[0]*len(volumes) for _ in range(len(volumes))]
  for i in range(len(volumes)):
    for j in range(i+1, len(volumes)):
      dist[i][j] = euclidean_distance(volumes[i], volumes[j])
      dist[j][i] = dist[i][j]
  return dist

## Absolute Distance

In [ ]:
def absolute_distance(value_i, value_j):
  return abs(value_i - value_j)

def get_abs_dist(values):
  dist = [[0]*len(values) for _ in range(len(values))]
  for i in range(len(values)):
    for j in range(i+1, len(values)):
      dist[i][j] = absolute_distance(values[i], values[j])
      dist[j][i] = dist[i][j]
  return dist

# Import Distances - SIFT3D

In [ ]:
import scipy.io as sio

def get_sift_dist():
  
  SiftDistFile = sio.loadmat('/content/mnt/MyDrive/Project_X/DistancesSift.mat')
  
  SiftDist = SiftDistFile['dist']
  
  return SiftDist

# IMED

In [ ]:
!pip install IMED

In [ ]:
'''
import IMED
from IMED.standardizingTrans_ndim import ST_ndim_DCT, ST_ndim_FFT, ST_ndim_DCT_by_FFT
from IMED.spatial_ST import ST_fullMat, ST_sepMat

#Helper function
def standardizingTrans(imgs,sigma,method='dct',eps=0,inverse=False):
    """
    Takes sequence of images imgs and returns the Spatial Standardized Transform of all images.
    Methods 'full' and 'sep' are 2D methods.

    Parameters:
    * imgs (3D array) is a sequence of images to be transformed with dimensions (T,M,N)
    * sigma (float)/array-like determines the zero-mean Gaussian that defines the IMED matrix G - not G^(1/2).
      If sigma is array-like it should contain the same number of values as the number of dimensions of imgs.

    * eps (float) is an optional small parameter to offset the Gaussian so that it is always numerically non-zero. 
    This can allow deconvolution without significant noise amplification.
    * method (string) is the method used to perform the standardizing transform. Choose between:
     1. **'full':** Full Dense Matrix $z_{ST}= G^{1/2}z$ using eigenvalue decomposition
     2. **'sep'** Separated Dense Matrices $z_{ST}= G_x^{1/2}z G_y^{1/2}$ using eigenvalue decomposition 
     3. **'fft'**: Performs circular convolution using discrete fourier transforms of image and Gaussian 
     without enforcing symmetric boundary conditions
     4. **'dct_by_fft'**: Performs circular convolution using discrete fourier transforms of mirrored image and 
     Gaussian to ensure symmetric boundary conditions and reduce edge effects from 'Gibbs-like phenomenon'
     5. **'dct'**: Performs symmetric convolution using discrete cosine transform of image, which is identical to
     6. the 'dct_by_fft  method, but should be more efficient
    """
    
    if  method == 'full':
        if inverse==True:
            print("No inverse method implemented")
            return
        imgs_ST = ST_fullMat(imgs,sigma,eps)
    
    elif method == 'sep':
        if inverse==True:
            print("No inverse method implemented")
            return
        imgs_ST = ST_sepMat(imgs,sigma,eps)

    elif method == 'fft':
        imgs_ST = ST_ndim_FFT(imgs, sigma, eps,inverse)
        
    elif method == 'dct_by_fft':
        imgs_ST = ST_ndim_DCT_by_FFT(imgs, sigma, eps,inverse)
    
    elif method == 'dct':
        imgs_ST = ST_ndim_DCT(imgs, sigma, eps, inverse)
        
    else:
        print(f'Invalid method "{method}". Choosing dct.')
        method = 'dct'
        standardizingTrans(imgs,sigma,method,eps,inverse)    
    
    return imgs_ST
'''

'\nimport IMED\nfrom IMED.standardizingTrans_ndim import ST_ndim_DCT, ST_ndim_FFT, ST_ndim_DCT_by_FFT\nfrom IMED.spatial_ST import ST_fullMat, ST_sepMat\n\n#Helper function\ndef standardizingTrans(imgs,sigma,method=\'dct\',eps=0,inverse=False):\n    """\n    Takes sequence of images imgs and returns the Spatial Standardized Transform of all images.\n    Methods \'full\' and \'sep\' are 2D methods.\n\n    Parameters:\n    * imgs (3D array) is a sequence of images to be transformed with dimensions (T,M,N)\n    * sigma (float)/array-like determines the zero-mean Gaussian that defines the IMED matrix G - not G^(1/2).\n      If sigma is array-like it should contain the same number of values as the number of dimensions of imgs.\n\n    * eps (float) is an optional small parameter to offset the Gaussian so that it is always numerically non-zero. \n    This can allow deconvolution without significant noise amplification.\n    * method (string) is the method used to perform the standardizing tra

In [ ]:
import numpy as np

def get_IMED_dist(volumes):

  new_volumes = volumes

  for i in range(len(volumes)):

    new_volumes[i] = standardizingTrans(np.array(volumes[i]), sigma = 1, method='fft', eps=0, inverse=False)

  return get_eucl_dist(new_volumes)

# WASSERSTEIN DISTANCE

In [ ]:
import scipy.stats
from scipy.stats import wasserstein_distance

In [ ]:
#Return the distance matrix between histograms. 
#Input = stat_features['Histogram'] or stat_features['Second Histogram']

def hist_dist(histograms): 

  dist = [[0]*len(histograms) for _ in range(len(histograms))]
  
  for i in range(len(histograms)):

    for j in range(i+1, len(histograms)):

      dist[i][j] = wasserstein_distance(histograms[i][0], histograms[j][0])

      dist[j][i] = dist[i][j]

  return dist

# **Distances for functions: AC distance, L1 distance for derivatives**

In [ ]:
## AC distance: distanza L1 delle funzioni + distanza L1 delle derivate 
## il codice funziona solo per funzioni lineari a tratti 

def get_AC_dist(values):
  dist = [[0]*len(values) for _ in range(len(values))]
  for i in range(len(values)):
    for j in range(i+1, len(values)):
      dist[i][j] = AC_dist(values.iloc[i], values.iloc[j])
      dist[j][i] = dist[i][j]
  return dist


import matplotlib.pyplot as plt
from scipy import interpolate


def AC_dist(f1,f2):
  # L1_ derivative dist
  L1_der_d = 0 
  for i in (range(len(f1)-1)):
    der_1 = (f1[i+1]-f1[i]) 
    der_2 = (f2[i+1]-f2[i])
    L1_der_d = L1_der_d + abs(der_1 - der_2)
  # L1 dist 
  L1_d = 0
  for i in range(len(f1)-1):
    if (f1[i]-f2[i])*(f1[i+1]-f2[i+1]) >= 0: 
      L1_d = L1_d + abs((f1[i]+f1[i+1])/2-(f2[i]+f2[i+1])/2)
    else :
      m1 = f1[i+1]-f1[i]
      m2 = f2[i+1]-f2[i]
      f1_line = interpolate.interp1d([i,i+1], [f1[i],f1[i+1]], kind = 'linear', fill_value = "extrapolate")
      f2_line = interpolate.interp1d([i,i+1], [f2[i],f2[i+1]], kind = 'linear', fill_value = "extrapolate")
      q1 = f1_line(0)
      q2 = f2_line(0)
      intersec = (q2 - q1 )/ ( m1 - m2 )
      h = intersec - i 
      L1_d = L1_d + abs(f1[i]-f2[i])*(h)/2+abs(f1[i+1]-f2[i+1])*(1-h)/2
  return L1_der_d + L1_d 

def L1_der_dist(f1,f2):
  # L1_ derivative dist
  L1_der_d = 0 
  for i in (range(len(f1)-1)):
    der_1 = f1[i+1]-f1[i]
    der_2 = f2[i+1]-f2[i]
    L1_der_d = L1_der_d + abs(der_1 - der_2)
  return L1_der_d
  
def get_L1_der_dist(values):
  dist = [[0]*len(values) for _ in range(len(values))]
  for i in range(len(values)):
    for j in range(i+1, len(values)):
      dist[i][j] = L1_der_dist(values.iloc[i], values.iloc[j])
      dist[j][i] = dist[i][j]
  return dist


